# Lab: Dimension reduction (2)

In the previous lab, we learned how to work with dimension reduction in a relatively simple dataset. In this lab we are going to work with a much larger dataset: a subset of the US Census & Crime data [@misc_communities_and_crime_183]. Details about the various variables can be found [here](http://archive.ics.uci.edu/ml/datasets/communities+and+crime).

How can we tackle a dataset with 100 different dimensions? We try using PCA and clustering. There is a lot you can do with this dataset. We encourage you to dig in. Try and find some interesting patterns in the data. Even upload your findings to the Teams channel and discuss.

## Data

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
df = pd.read_csv('data/censusCrimeClean.csv')

df

::: callout-caution

## Do-this-yourself: Let's say "hi" to the data

You have probably learnt by now that the best start to working with a new data set is to get to know it. What would be a good way to approach this?

A few steps we always take:
- Use the internal spreadsheet viewer in JupyterLab, go into the data folder and have a look at the data.
- Check the size of the dataset. Do you remember how to look at the `shape` of a dataset?
- Get a summary of the data, observe the *descriptive* statistics.
- Check the data types and see if there are mixes of different types, such as numerical, categorical, textual, etc..
- Create a visualisation to get a first peek into the relations. Think about the number of features. Is a visualisation feasible?
  

::: callout-caution

## Do-this-yourself: Check if we need to do any normalisation for this case?

What are the descriptive statistics look like, see if we need to do anything more?

Usually a standardisation is recommended before you apply PCA on a dataset. Here is that link again -- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

## Principal Component Analysis


**Note for the above:** In case you wondered, we have done some cleaning and preprocessing on this data to make things easier for you, so normalisation might not be needed or at least won't make a lot of difference to the results. But the real life will always be much more messier.

OK, you probably have noticed that there is a lot of data features we have here. Let's start experimenting with the PCA technique and see what it could surface. Maybe it will help us reduce the dimensionality a little but and tell us something about the features.

In [ ]:
from sklearn.decomposition import PCA

n_components = 2
 
pca = PCA(n_components=n_components)
df_pca = pca.fit(df.iloc[:, 1:])

In [ ]:
df_pca.explained_variance_ratio_

In [ ]:
::: callout-caution

## What do these numbers mean?

In [ ]:
df_pca_vals = pca.fit_transform(df.iloc[:,1:])
df['c1'] = [item[0] for item in df_pca_vals]
df['c2'] = [item[1] for item in df_pca_vals]

In [ ]:
import seaborn as sns
sns.scatterplot(data = df, x = 'c1', y = 'c2')

::: callout-caution

Hmm, that looks problematic. What is going on? When you see patterns like this, you will need to take a step back and look at everything closely.

We should look at the component loadings. Though there are going to be over 200 of them. We can put them into a Pandas dataframe and then sort them.

In [ ]:
data = {'columns' : df.columns[1:102],
        'component 1' : df_pca.components_[0],
        'component 2' : df_pca.components_[1]}


loadings = pd.DataFrame(data)
loadings.sort_values(by=['component 1'], ascending=False, key=abs) 

In [ ]:
loadings.sort_values(by=['component 2'], ascending=False) 

::: callout-caution

Do you notice anything extraordinary within these variable loadings?

Remember, the variable loadings tell us to what extent the information encoded along a principal component axis is determined by a single variable. In other words, how important a variable is. Haev a look and see if anything is too problematic here.




The fold variable is messing with our PCA. What does that variable look like.

In [ ]:
df.fold.value_counts()

In [ ]:
df['fold'].unique()

Aha! It looks to be some sort of categorical variable. Looking into the dataset more, it is actually a variable used for cross tabling.

We should not include this variable in our analysis.

In [ ]:
df.iloc[:, 2:-2]

In [ ]:
from sklearn.decomposition import PCA

n_components = 2

pca_no_fold = PCA(n_components=n_components)
df_pca_no_fold = pca_no_fold.fit(df.iloc[:, 2:-2])

df_pca_vals = pca_no_fold.fit_transform(df.iloc[:, 2:-2])

df['c1_no_fold'] = [item[0] for item in df_pca_vals]
df['c2_no_fold'] = [item[1] for item in df_pca_vals]

sns.scatterplot(data = df, x = 'c1_no_fold', y = 'c2_no_fold')

How do our component loadings look?

In [ ]:
data = {'columns' : df.iloc[:, 2:-4].columns,
        'component 1' : df_pca_no_fold.components_[0],
        'component 2' : df_pca_no_fold.components_[1]}


loadings = pd.DataFrame(data)
loadings_sorted = loadings.sort_values(by=['component 1'], ascending=False)
loadings_sorted.iloc[1:10,:]

In [ ]:
loadings_sorted = loadings.sort_values(by=['component 2'], ascending=False)
loadings_sorted.iloc[1:10,:]

Interesting that our first component variables are income related whereas our second component variables are immigration related. If we look at the projections of the model, coloured by Crime, what do we see?

In [ ]:
sns.scatterplot(data = df,
                x = 'c1_no_fold',
                y = 'c2_no_fold',
                hue = 'ViolentCrimesPerPop',
                size = 'ViolentCrimesPerPop')

## Clustering

What about clustering using these `income` and `immigration` components?

### Defining number of clusters

In [ ]:
from sklearn.cluster import KMeans

ks = range(1, 10)
inertias = []
for k in ks:
    # Create a KMeans instance with k clusters: model
    model = KMeans(n_clusters=k, n_init = 10)
    
    # Fit model to samples
    model.fit(df[['c1_no_fold', 'c2_no_fold']])
    
    # Append the inertia to the list of inertias
    inertias.append(model.inertia_)

import matplotlib.pyplot as plt

plt.plot(ks, inertias, '-o', color='black')
plt.xlabel('number of clusters, k')
plt.ylabel('inertia')
plt.xticks(ks)
plt.show()

Four clusters looks good.

## Visualising clusters

In [ ]:
k_means_4 = KMeans(n_clusters = 3, init = 'random', n_init = 10)
k_means_4.fit(df[['c1_no_fold', 'c2_no_fold']])
df['Four clusters'] = pd.Series(k_means_4.predict(df[['c1_no_fold', 'c2_no_fold']].values), index = df.index)

In [ ]:
sns.scatterplot(data = df, x = 'c1_no_fold', y = 'c2_no_fold', hue = 'Four clusters')

Hmm, might we interpret this plot? The plot is unclear. Let us assume c1 is an income component and c2 is an immigration component. Cluster 0 are places high on income and low on immigration. Cluster 2 are low on income and low on immigration. The interesting group are those high on immigration and relatively low on income.

We can include crime on the plot.

In [ ]:
#| column: screen

import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (15,10)
sns.scatterplot(data = df, x = 'c1_no_fold', y = 'c2_no_fold', hue = 'Four clusters', size = 'ViolentCrimesPerPop')

Alas, we stop here in this exercise. You could work on this further. There are outliers you might want to remove. Or, at this stage, you might want to look at more components, 3 or 4 and look for some other factors.